In [1]:
import pandas as pd
import requests
import lxml
from lxml import html
import time
import json
from tqdm.notebook import tqdm

In [2]:
from utilities import *

In [3]:
words = pd.read_excel('Words from GRE Vocabulary Assessment Videos.xlsx', header=None)
words = words.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [4]:
max_groups = 28
max_set_words = 30

In [5]:
# # Create new
ankified = pd.DataFrame(columns=['words','definition','synonyms','mnemonics','image_url'])

# ankified = pd.read_csv('anki_mnemonics.csv')

In [9]:
ankified

,words,definition,synonyms,mnemonics,image_url,Set
0,abound,(verb) be abundant or plentiful; exist in larg...,"bristle,burst",Mnemonic 1: Bounded means limited. Abound mean...,[],Set 1
1,amorphous,(adj) having no definite form or distinct shape,"formless,shapeless,unstructured,uncrystallised...",Mnemonic 1: In Greek 'morphe' means shape/form...,[],Set 1
2,austere,(adj) severely simple,"severe,stark,stern,stern,ascetic,ascetical,spa...",Mnemonic 1: AUS-australia TERE-tear(sorrow). l...,[],Set 1
3,belie,(verb) be in contradiction with,"contradict,negate,misrepresent",Mnemonic 1: belie ~ lie means something false....,[],Set 1
4,capricious,(adj) changeable,"freakish,impulsive,whimsical",Mnemonic 1: ca(CAR) + PRIC(PRICE)...PRICE OF c...,[],Set 1
5,cerebral,(adj) involving intelligence rather than emoti...,intellectual,Mnemonic 1: cereBRAINal,[],Set 1
6,congenial,(adj) suitable to your needs,,Mnemonic 1: its con +genial.... so remember ge...,[https://dailyvocab.com/wp-content/uploads/201...,Set 1
7,conspicuous,(adj) obvious to the eye or mind,"blatant,blazing",Mnemonic 1: Con(can)+ s(see) + the + pic(pictu...,[],Set 1
8,cursory,(adj) hasty and without attention to detail; n...,"casual,passing,perfunctory",Mnemonic 1: relate it to the cursor on your co...,[],Set 1
9,daunting,(adj) discouraging through fear,intimidating,,[https://dailyvocab.com/wp-content/uploads/201...,Set 1


In [7]:
# ankified['mnemonics'][2].strip('[]')
# ankified['mnemonics'] = ankified['mnemonics'].apply(lambda x: None if x == '[]' else x)
# ankified['mnemonics'] = ankified['mnemonics'].apply(lambda x: x.strip('[]').split(',') if x != None else None) # String to list
# ankified['mnemonics'] = ankified['mnemonics'].apply(lambda x: mnemo_list_to_string(x) if x != 'NaN' else x) # list to mnemonics

In [ ]:
total_sets = 28
for set_no in range(1, total_sets+1):
    word_list = list(get_my_set(words, set_no))
    ankified = add_set(ankified, word_list, set_no, sleep = True)

In [ ]:
ankified

In [ ]:
ankified['image_url'] = ankified['image_url'].apply(lambda x: x.strip('[]\'') if isinstance(x, str) else x) # Removing '[', ']', ''' (comma) from omage_url
ankified['image_url'] = ankified['image_url'].apply(lambda x: None if isinstance(x, list) and len(x)==0 else x ) # Removing empty lists from url
ankified['image_url'] = ankified['image_url'].apply(lambda x: x[0] if isinstance(x, list) else x ) # list to string
ankified['image_url'] = ankified['image_url'].apply(lambda x: '<img src="{}">'.format(x) if isinstance(x, str) else None)

In [ ]:
ankified['image_url'].value_counts()

In [ ]:
ankified.to_csv('anki_mnemonics.csv', index = None)

# Requests and BS4

In [172]:
page = requests.get(url)
page.status_code

200

In [175]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page.text, 'html.parser')

print(soup.prettify())

soup.find_all('p')

soup.find_all('p')[2].get_text()

## Testing Section

In [38]:
word = 'abjure'

In [39]:
url = 'https://mnemonicdictionary.com/?word='+ word

page = requests.get(url)
tree = html.fromstring(page.content)
definition = tree.xpath('//li[@class="media list-group-item p-4"]/div[1]//text()')[12].strip()

In [40]:
tree.xpath('//li[@class="media list-group-item p-4"]/div/div//text()')[10]

' (verb) formally reject or disavow a formerly held belief, usually under pressure\n                              '

['forswear', 'recant', 'resile', 'retract']

In [14]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page.text, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="MnemonicDictionary.com - Meaning of quixotic and a memory aid (called Mnemonic) to retain that meaning for long time in our memory." name="description"/>
  <meta content="quixotic, meaning, definition, word power made easy, mnemonic, mnemonics, dictionary, gre words, gre dictionary, vocabulary, word list" name="keywords"/>
  <meta content="OpenCubicles" name="author">
   <title>
    quixotic meaning - definition of quixotic by Mnemonic Dictionary
   </title>
   <link href="https://fonts.googleapis.com/css?family=Open+Sans:400,300,600" rel="stylesheet" type="text/css"/>
   <link href="/assets/css/toolkit.css" rel="stylesheet"/>
   <link href="/assets/css/application.css" rel="stylesheet"/>
   <link crossorigin="anonymous" href="https://use.fontawesome.com/releases/v5.0.13/css/a

In [ ]:
soup.find_all('p')

soup.find_all('p')[2].get_text()